In [1]:
### NOTES ###
# MAKE SURE TO RUN ALL CELLS
# AUTHORS: Jack, Andrew, Lauren
# VERSION: 1.0 (final)

import os
from numbers import Number
from typing import List, Any, Dict, Callable, Union, Iterable

import numpy as np
from IPython.lib.display import Audio
from numpy import ndarray, dtype
from scipy.signal import sawtooth, square, gausspulse
from ipywidgets import widgets, interact, IntSlider, FloatSlider, ToggleButton, Dropdown

### CONSTANTS ###
NOTES = {b: a for a, b in enumerate(('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'))}
NOTES.update({b: a for a, b in enumerate(('B#', 'Db', 'D', 'Eb', 'Fb', 'E#', 'Gb', 'G', 'Ab', 'A', 'Bb', 'Cb'))})
REST = "X"
DEFAULT_DROPDOWN = "Select an option"
SHAPES = {'saw': sawtooth, 'square': square, 'pulse': gausspulse, 'sinusoid': np.sin}

In [2]:
class Synth:
    width: float
    shape: Callable
    fm: dict[str, float]
    delay: dict[str, float]
    adsr: list[float]
    rev: bool
    bpm: int
    length: int
    file: str
    result: ndarray[Any, dtype[Any]]
    sequence: list[tuple[tuple[float, ...], float]]
    global_rate: int

    def __init__(self, global_rate=48000):
        # Set all defaults here
        self.file = ''
        self.global_rate = global_rate
        self.sequence = []
        self.length = 0
        self.result = np.array([])
        self.bpm = 120
        self.rev = False
        self.adsr = [0, 0, 1, 0]
        self.delay = dict(mix=0, length=0, count=0)
        self.fm = dict(index=2.234)
        self.shape = sawtooth 
        self.width = 0.5
        
        
    def set_sequence(self, file_path: str):
        """
        Parses a file containing notes and durations and sets the sequence for playback.
        
        Stores it in a sequence variable which is formatted like so:
            [([NOTE_A, NOTE_B, ...], NOTE_LENGTH), ...]
        
        :param file_path: Path to a txt file containing the sequence information.
        """
        if not file_path or file_path == DEFAULT_DROPDOWN:
            return
        
        # Reset score
        self.sequence = [] 
        self.file = file_path
        self.length = 0
        with open(file_path, 'r') as file:
            for line in file:
                # Split each line
                parts = line.strip().split()
                self.sequence.append((tuple(self.note_to_freq(note) for note in parts[:-1]),  240 / self.bpm * float(parts[-1])))
                # Add the length of the last added item to the total length (for later use)
                self.length += self.sequence[-1][1]
        # print(f"New length {self.length} based on bpm={self.bpm}")
    
    ### The following methods are simply setters that interface with the UI
    
    def set_shape(self, shape: str):
        self.shape = SHAPES[shape]
    
    def set_width(self, width: float):
        self.width = width
    
    def set_fm(self, key, val: float):
        if key in self.fm:
            self.fm[key] = val
    
    def set_bpm(self, bpm: int):
        self.bpm = bpm
        # Have to reset timings
        self.set_sequence(self.file)
    
    def set_reversed(self, rev: bool):
        self.rev = rev
    
    def set_delay(self, key, val: float):
        if key in self.delay:
            self.delay[key] = val
    
    def set_adsr(self, *args: list[float]):
        if len(args) == 4:
            # print("new ADSR", self.adsr)
            self.adsr = args
        else:
            # print("old ADSR", self.adsr)
            pass

    @staticmethod
    def note_to_freq(note: str) -> float:
        """
        Converts a string representation of a note to a floating point frequency.
        
        :param note: note in the form [CHARACTER(S)][OCTAVE] e.g. C#4
        :return: frequency associated with MIDI value
        """
        if note == REST:
            return 0
        # Get the number associated with the scale degree
        letter = NOTES[note[:-2] if note.find('-') > -1 else note[:-1]]
        # Get the octave (number) associated with the string
        octave = int(note[-2:] if note.find('-') > -1 else note[-1:])
        # Offset the pitch appropriately
        pitch = 12 * (octave + 1) + letter
        # MIDI to frequency
        return 440 * 2 ** ((pitch - 69) / 12)

    def calculate_env(self, dur: float, a: float, d: float, s: float, r: float) -> np.ndarray:
        """
        Calculates the shape of the envelope for the individual note passed in.
        
        Fits to size by extending/trimming the sustain level to fit the note length when necessary. 
        
        :param dur: duration of the note (seconds)
        :param a: attack (in seconds)
        :param d: delay (in seconds)
        :param s: sustain level
        :param r: release
        :return: an array representing the shape of the envelope
        """
        samples = int(self.global_rate * dur)
        attack = np.linspace(0, 1, int(self.global_rate * a))
        decay = np.linspace(1, s, int(self.global_rate * d))
        if dur < a + d:
            env = np.concatenate((attack, decay))
            release = np.linspace(env[-1], 0, int(self.global_rate*r))
            env = np.concatenate((env[:samples], release))
        else:
            sussy = np.full(int((dur - a - d) * self.global_rate), s)
            release = np.linspace(s, 0, int(self.global_rate*r))
            env = np.concatenate([attack, decay, sussy, release])
        return env

    def delay_audio(self, audio, ms: int, mix: float, delays: int) -> np.ndarray:
        """
        Delays the audio signal if elected.
        
        :param audio: input signal
        :param ms: length to delay each voice by
        :param mix: % of output signal that is wet
        :param delays: number of voices
        :return: the output signal (w/ delay mixed in)
        """
        
        # Conversion to samples
        sec = ms / 1000.0
        samples = int(delays * sec * self.global_rate)
        pad = np.zeros(samples)
        dry = np.concatenate((audio, pad), axis=0)
        wet = np.zeros(dry.size)
        for delay in range(delays):
            # Add each delayed signal to the wet result
            wet += np.roll(dry, (delay+1)*int(sec*self.global_rate))/(1+delay)
        # Calculates the mix between wet and dry signals
        output = (1-mix)*dry + mix*wet
        return output

    def _bell_base_generator(self) -> np.ndarray:
        """
        Private method that performs all the calculations in series.
        
        :return: the resulting synth sound, as an array
        """
        # Get ADSR values
        attack, decay, sustain, release = tuple(self.adsr)
        # Get delay values
        ms, delays, mix = self.delay['length'], self.delay['count'], self.delay['mix']
        # Get FM values
        index = self.fm['index']
        # Useful variable for timing purposes
        dsr = [decay, sustain, release]
        curr_samples = 0
        pre_processed = np.zeros(int(self.global_rate * (self.length + dsr[2])))

        # Go through each series of notes
        for chord in self.sequence:
            # Total duration of the chord determined by chord length + release
            dur = chord[1] + dsr[2]
            da_chord = np.zeros(int(self.global_rate * dur))
            t = np.linspace(0, dur, int(self.global_rate * dur))
            for freq in chord[0]:
                if freq == 0:
                    # This means there is a rest
                    da_chord += np.zeros(int(self.global_rate * (chord[1] + dsr[2])))
                    break
                mod = index
                
                # Sinusoid don't allow for PWM thus 2 arguments won't work every time
                try:
                    mod *= self.shape(2 * np.pi * freq * 2 * (t + 1 / (freq * 8)), self.width)
                except TypeError:
                    self.shape(2 * np.pi * freq * 2 * (t + 1 / (freq * 8)))
                 
                try:
                    fm = self.shape(2 * np.pi * freq * (t + 1 / (freq * 4)) + mod, self.width)
                except TypeError:
                    fm = self.shape(2 * np.pi * freq * (t + 1 / (freq * 4)) + mod)
                da_chord += fm
            
            # Get the envelope associated with this chord
            env = self.calculate_env(chord[1], attack, *dsr)
            
            # Aligns envelope array if there is an off-by-one error (due to integer flooring ¯\_(ツ)_/¯)
            diff1 = len(env) - len(da_chord)
            if diff1 > 0:
                env = env[diff1:]
            elif diff1 < 0:
                env = np.concatenate((np.zeros(-diff1), env))
            env *= da_chord
            pad = np.concatenate((env, np.zeros(int(int(self.length * self.global_rate) - self.global_rate * chord[1]))))
            rolled = np.roll(pad, curr_samples)
            curr_samples += int(chord[1] * self.global_rate)
            
            # See above comment
            diff = len(pre_processed) - len(rolled)
            if diff == 1:
                rolled = np.concatenate((np.array([0]), rolled))
            elif diff == -1:
                rolled = rolled[1:]
            pre_processed += rolled
        
        # Delay the audio
        post_processed = self.delay_audio(pre_processed, ms, mix=mix, delays=delays) 
        
        # Reverse the audio
        if self.rev:
            post_processed = post_processed[::-1]
        
        # Return the final audio after processing
        return post_processed
    
    def generate_audio(self) -> Audio:
        """
        This method interfaces with the UI to indicate when to request a new audio file.
        
        :return: The output audio object
        """
        if self.file:
            return Audio(self._bell_base_generator(), rate=self.global_rate)
        else:
            print("File has not been selected.")
        

In [3]:
class SynthUI:
    activate_button: ToggleButton
    file_dropdown: Dropdown
    index_slider: FloatSlider
    width_slider: FloatSlider
    shape_dropdown: Dropdown
    del_mix_slider: FloatSlider
    del_count_slider: IntSlider
    del_length_slider: IntSlider
    rev_toggle: ToggleButton
    adsr_sliders: list[FloatSlider]
    bpm_slider: IntSlider
    text_files: list[str]
    synth: Synth

    def __init__(self, synth):
        self.synth = synth
        self.text_files = list(file for file in os.listdir('.') if file[-4:] == '.txt')
        
        # BPM
        self.bpm_slider = widgets.IntSlider(
            value=self.synth.bpm,
            min=60,
            max=240,
            step=1,
            description='BPM:',
            continuous_update=False
        )
        
        # Envelope
        self.adsr_sliders = [
            widgets.FloatSlider(
                value=getattr(self.synth, 'adsr')[i],
                min=0,
                max=val,
                step=0.01,
                description=label,
                continuous_update=False
            ) for i, (label, val) in enumerate([('Attack:', 1), ('Decay:', 5), ('Sustain:', 1), ('Release:', 5)])
        ]

        # Reversed FX
        self.rev_toggle = widgets.ToggleButton(
            value=self.synth.rev,
            description='Reverse Audio',
            tooltip='Reverse the sequence'
        )        
        
        # Delay
        self.del_length_slider = widgets.IntSlider(
            value=self.synth.delay['length'],
            min=0,
            max=100,
            step=1,
            description='Delay (ms):',
            continuous_update=False
        )
        
        self.del_count_slider = widgets.IntSlider(
            value=self.synth.delay['count'],
            min=0,
            max=10,
            step=1,
            description='Delay #:',
            continuous_update=False
        )
        
        self.del_mix_slider = widgets.FloatSlider(
            value=self.synth.delay['mix'],
            min=0,
            max=1,
            step=0.01,
            description='Delay Mix:',
            continuous_update=False
        )
        
        # Shape Selection
        self.shape_dropdown = widgets.Dropdown(
            options=SHAPES.keys(),
            value='saw',
            description='Shape:',
        )
        
        # PWM
        self.width_slider = widgets.FloatSlider(
                value=getattr(self.synth, 'width'),
                min=0,
                max=1,
                step=0.01,
                description='Pulse Width',
                continuous_update=False
            ) 
        
        # FM
        self.index_slider = widgets.FloatSlider(
                value=getattr(self.synth, 'fm')['index'],
                min=0,
                max=10,
                step=0.0001,
                description='Index',
                continuous_update=False
            ) 
        
        # File Selection
        self.file_dropdown = widgets.Dropdown(
            options=[DEFAULT_DROPDOWN] + self.text_files,
            value=DEFAULT_DROPDOWN,
            description='Score file:',
        )
        
        # Button to active everything
        self.activate_button = widgets.ToggleButton(value=False, description="Generate audio")
        
        # Display the UI - remains in this state for the remainder of the program
        self.display_ui()
        
    def display_ui(self):
        """
        Displays all the components to the user, allowing for manipulation.
        """
        
        display(widgets.Label("Sound"))
        interact(self.synth.set_shape, shape=self.shape_dropdown)
        interact(self.synth.set_width, width=self.width_slider)
        interact(self.update_fm_index, val=self.index_slider)
        
        display(widgets.Label("Envelope"))
        for slider in self.adsr_sliders:
            interact(self.update_adsr, active=slider)
            
        display(widgets.Label("FX"))
        interact(self.synth.set_reversed, rev=self.rev_toggle)
        interact(self.update_del_ms, val=self.del_length_slider)
        interact(self.update_del_count, val=self.del_count_slider)
        interact(self.update_del_mix, val=self.del_mix_slider)
        
        display(widgets.Label("Score"))
        interact(self.synth.set_bpm, bpm=self.bpm_slider)
        interact(self.synth.set_sequence, file_path=self.file_dropdown)
        interact(self.activate, state=self.activate_button)
    
    
    ### The following methods are intermediary to the synth class because interact is just weird like that
    
    def update_fm_index(self, val):
        self.synth.set_fm('index', val)
        
    def update_del_ms(self, val):
        self.synth.set_delay('length', val)
    
    def update_del_count(self, val):
        self.synth.set_delay('count', val)
        
    def update_del_mix(self, val):
        self.synth.set_delay('mix', val)
    
    def update_adsr(self, active):
        if active:
            self.synth.set_adsr(*[slider.value for slider in self.adsr_sliders])
    
    def activate(self, state):
        if state:
            res = self.synth.generate_audio()
            if res:
                display(res)
            # self.activate_button.value = False


In [4]:
# THIS CELL INITS THE SYNTH
synth = Synth()
synth_ui = SynthUI(synth)

Label(value='Sound')

interactive(children=(Dropdown(description='Shape:', options=('saw', 'square', 'pulse', 'sinusoid'), value='sa…

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='Pulse Width', max=1.0, step…

interactive(children=(FloatSlider(value=2.234, continuous_update=False, description='Index', max=10.0, step=0.…

Label(value='Envelope')

...

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Delay Mix:', max=1.0, step=…

Label(value='Score')

interactive(children=(IntSlider(value=120, continuous_update=False, description='BPM:', max=240, min=60), Outp…

interactive(children=(Dropdown(description='Score file:', options=('Select an option', 'demo.txt', 'sean.txt',…

interactive(children=(ToggleButton(value=False, description='Generate audio'), Output()), _dom_classes=('widge…

## How to use:
1. Edit parameters
2. Click generate to hear the result
3. To update with new parameters, double-click generate